<a href="https://colab.research.google.com/github/changhorang/SSAC_study/blob/main/DL3%20(PyTorch)/%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC/Chapter5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 문장 쌍 분류하기


## 5-1 문장 쌍 분류 모델 훑어보기
- 문장 쌍 분류(sentence pair classification)의 대표 예로 자연어 추론(natural language inference, NLI) 실습
- 자연어 추론: 2개의 문장 또는 문서가 참(entailment), 거짓(contradiction), 중립 또는 판단 불가(neutral)인 지 가려내는 것
- 업스테이지에서 공개한 NLI 데이터셋을 이용 (전제, 가설, 레이블로 구성)

### 모델
- input: [cls] + 토큰화된 전제 + [sep] + 토큰화된 가설 + [sep]
- BERT 모델 => pooler_output (prediction)

### 태스크 모듈
- pooler_output에 Dropout 적용 후 3차원 벡터로 적용 (class_number)
- softmax를 취해 최종 출력과 정답 레이블이 같아지도록 업데이트
- pooler_output벡터에는 2개의 의미가 내포되어짐

## 5-2 문장 쌍 분류 모델 학습하기
### 전제와 가설을 검증하는 자연어 추론 모델 만들기
#### 각종 설정하기 (구글 드라이브 마운트 포함)

In [1]:
!pip install ratsnlp

     |████████████████████████████████| 53 kB 2.0 MB/s 
     |████████████████████████████████| 806 kB 34.4 MB/s 
     |████████████████████████████████| 2.8 MB 58.9 MB/s 
     |████████████████████████████████| 57 kB 7.0 MB/s 
     |████████████████████████████████| 132 kB 60.2 MB/s 
     |████████████████████████████████| 332 kB 72.7 MB/s 
     |████████████████████████████████| 829 kB 60.3 MB/s 
     |████████████████████████████████| 636 kB 66.8 MB/s 
     |████████████████████████████████| 61 kB 674 kB/s 
     |████████████████████████████████| 895 kB 69.2 MB/s 
     |████████████████████████████████| 3.3 MB 54.4 MB/s 
     |████████████████████████████████| 1.1 MB 60.7 MB/s 
     |████████████████████████████████| 96 kB 7.2 MB/s 
     |████████████████████████████████| 160 kB 70.8 MB/s 
     |████████████████████████████████| 192 kB 73.8 MB/s 
     |████████████████████████████████| 271 kB 74.7 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 

In [2]:
import torch
from ratsnlp.nlpbook.classification import ClassificationTrainArguments

args = ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base',
                                    downstream_task_name='pair-classification',
                                    downstream_corpus_name='klue-nli',
                                    downstream_model_dir='/content/drive/MyDrive/nlpbook/checkpoint-paircls',
                                    batch_size=32 if torch.cuda.is_available() else 4,
                                    learning_rate=5e-5,
                                    max_seq_length=64,
                                    epochs=5,
                                    tpu_cores=0 if torch.cuda.is_available() else 8,
                                    seed=7)

# random seed 고정
from ratsnlp import nlpbook
nlpbook.set_seed(args)

# logger 설정
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='pair-classification', downstream_corpus_name='klue-nli', downstream_corpus_root_dir='/root/Korpora', downstream_model_dir='/content/drive/MyDrive/nlpbook/checkpoint-paircls', max_seq_length=64, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=5, batch_size=32, cpu_workers=2, fp16=False, tpu_cores=0)


set seed: 7


#### 말뭉치 내려받기 (root_dir(/root/Korpora)에 저장)

In [3]:
nlpbook.download_downstream_dataset(args)

Downloading: 100%|██████████| 12.3M/12.3M [00:00<00:00, 43.3MB/s]
Downloading: 100%|██████████| 1.47M/1.47M [00:00<00:00, 42.1MB/s]


#### 토크나이저 준비

In [4]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained(args.pretrained_model_name, do_lower_case=False)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

#### 데이터 전처리하기

In [5]:
from ratsnlp.nlpbook.paircls import KlueNLICorpus
from ratsnlp.nlpbook.classification import ClassificationDataset

corpus = KlueNLICorpus()
train_dataset = ClassificationDataset(args=args,
                                      corpus=corpus,
                                      tokenizer=tokenizer,
                                      mode='train')

INFO:ratsnlp:Creating features from dataset file at /root/Korpora/klue-nli
INFO:ratsnlp:loading train data... LOOKING AT /root/Korpora/klue-nli/klue_nli_train.json
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 11.882 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence A, B: 100분간 잘껄 그래도 소닉붐땜에 2점준다 + 100분간 잤다.
INFO:ratsnlp:tokens: [CLS] 100 ##분간 잘 ##껄 그래도 소 ##닉 ##붐 ##땜에 2 ##점 ##준다 [SEP] 100 ##분간 잤 ##다 . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
INFO:ratsnlp:label: contradiction
INFO:ratsnlp:features: ClassificationFeatures(input_ids=[2, 8327, 15760, 2483, 4260, 8446, 1895, 5623, 5969, 10319, 21, 4213, 10172, 3, 8327, 15760, 2491, 4020, 17, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [6]:
train_dataset[0]

ClassificationFeatures(input_ids=[2, 8327, 15760, 2483, 4260, 8446, 1895, 5623, 5969, 10319, 21, 4213, 10172, 3, 8327, 15760, 2491, 4020, 17, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], label=1)

In [7]:
# 학습 데이터 로더 구축
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

train_dataloader = DataLoader(train_dataset,
                              batch_size=args.batch_size,
                              sampler=RandomSampler(train_dataset, replacement=False),
                              collate_fn=nlpbook.data_collator,
                              drop_last=False,
                              num_workers=args.cpu_workers)

# validation dataset & dataloader
val_dataset = ClassificationDataset(args=args,
                                      corpus=corpus,
                                      tokenizer=tokenizer,
                                      mode='test')

val_dataloader = DataLoader(val_dataset,
                              batch_size=args.batch_size,
                              sampler=SequentialSampler(val_dataset),
                              collate_fn=nlpbook.data_collator,
                              drop_last=False,
                              num_workers=args.cpu_workers)

INFO:ratsnlp:Creating features from dataset file at /root/Korpora/klue-nli
INFO:ratsnlp:loading test data... LOOKING AT /root/Korpora/klue-nli/klue_nli_dev.json
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 1.175 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence A, B: 10명이 함께 사용하기 불편함없이 만족했다. + 10명이 함께 사용하기 불편함이 많았다.
INFO:ratsnlp:tokens: [CLS] 10명 ##이 함께 사용 ##하기 불편 ##함 ##없이 만족 ##했다 . [SEP] 10명 ##이 함께 사용 ##하기 불편 ##함이 많았 ##다 . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
INFO:ratsnlp:label: contradiction
INFO:ratsnlp:features: ClassificationFeatures(input_ids=[2, 21000, 4017, 9158, 9021, 8268, 10588, 4421, 8281, 14184, 8258, 17, 3, 21000, 4017, 9158, 9021, 8268, 10588, 11467, 14338, 4020, 17, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

#### 모델 불러오기

In [8]:
from transformers import BertConfig, BertForSequenceClassification

pretrained_model_config = BertConfig.from_pretrained(args.pretrained_model_name,
                                                     num_labels=corpus.num_labels)

model = BertForSequenceClassification.from_pretrained(args.pretrained_model_name,
                                                      config=pretrained_model_config)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

#### 모델 학습

In [10]:
from ratsnlp.nlpbook.classification import ClassificationTask

# task 정의
task = ClassificationTask(model, args)

# trainer 정의
trainer = nlpbook.get_trainer(args)

# 학습
trainer.fit(task, train_dataloader=train_dataloader, val_dataloaders=val_dataloader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClassification | 108 M 
--------------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
435.683   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

## 5-3 학습 마친 모델 실전 투입
### 전제와 가설을 검증하는 웹서비스 만들기

In [12]:
from ratsnlp.nlpbook.classification import ClassificationDeployArguments

args = ClassificationDeployArguments(pretrained_model_name='beomi/kcbert-base',
                                    downstream_model_dir='/content/drive/MyDrive/nlpbook/checkpoint-paircls',
                                    max_seq_length=64)

downstream_model_checkpoint_fpath: /content/drive/MyDrive/nlpbook/checkpoint-paircls/epoch=1-val_loss=0.84.ckpt


#### 토크나이저 및 모델 불러오기

In [13]:
from transformers import BertTokenizer
import torch

# tokenizer load
tokenizer = BertTokenizer.from_pretrained(args.pretrained_model_name, do_lower_case=False)

# checkpoint load
fine_tuned_model_ckpt = torch.load(args.downstream_model_checkpoint_fpath, map_location=torch.device('cuda'))

In [14]:
from transformers import BertConfig
from transformers import BertForSequenceClassification

# BERT 설정 로드
pretrained_model_config = BertConfig.from_pretrained(args.pretrained_model_name,
                                                     num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel())

# BERT 모델 초기화
model = BertForSequenceClassification(pretrained_model_config)

# checkpoint load
model.load_state_dict({k.replace('model.', ''): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})

# eval
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

#### 모델 출력값 만들고 후처리

In [ ]:
# inference 함수
def inference_fn(premise, hypothesis):
    